In [25]:
import os
import matplotlib.pyplot
import pandas as pd
import json
from glob import glob
from tqdm import tqdm
import requests
import subprocess
import yt_dlp
tqdm.pandas()

In [26]:
csv_path = "D:\projects\ProfanityFilter\explicits\songs_info_explicit.csv"
csv_expl_path = "D:\projects\ProfanityFilter\explicits\explicits_with_two_versions.txt"
df_main = pd.read_csv(csv_path)
df_explicit = pd.read_csv(csv_expl_path, names=["uri", "has_clean", "clean_uri", "expl_dur", "clean_dur"])

In [27]:
df = pd.merge(df_main, df_explicit, right_on="uri", left_on="uri", how='outer')

In [28]:
df[df.has_clean==True].shape

(13235, 11)

In [29]:
df_clean = df[df.has_clean==True]
df_clean.reset_index(drop=True, inplace=True)
last_uri = open("D:\projects\ProfanityFilter\explicits\youtube_links_new.txt").read().splitlines()[-1].split(",")[0]
start = df_clean[df_clean.uri==last_uri].index.item()
df_clean = df_clean[start+1:]
start

13234

In [30]:
df_clean

,uri,title,artist,explicit,popularity,isrc,artists_uri,has_clean,clean_uri,expl_dur,clean_dur


In [31]:
def save_youtube_info(*data):
    with open("D:\projects\ProfanityFilter\explicits\youtube_links_new.txt", "a", encoding="utf-8") as wf:
        line = ','.join(list(map(str, data)))
        wf.write(line +'\n')

In [32]:
params1 = ""
params2 = "intitle:clean"
yt_dlp_params = {
    "quiet": True,
    "retries": 10,
#     "age_limit": 16,
    "ignoreerrors":True,
}
with yt_dlp.YoutubeDL(yt_dlp_params) as yt:

    for i, row in tqdm(df_clean.iterrows(), total=len(df_clean)):
        artist = row.artist
        title = row.title
        duration = row.expl_dur / 1000
        q1 = f'ytsearch1:intitle:"{title}" {artist}, audio {params1}'
        q2 = f'ytsearch3:intitle:"{title}" {artist}, audio {params2}'
        r1 = yt.extract_info(q1, download=False)
        expl_id, expl_title, expl_dur = '', '', duration
        if "entries" in r1:
            for r in r1["entries"]:
                if r["duration"] < 600:
                    expl_id, expl_title, expl_dur = r["id"], r["title"], r["duration"]
        clean_id, clean_title, clean_dur = '', '', ''
        r2 = yt.extract_info(q2, download=False)
        if "entries" in r2:
            for r in r2["entries"]:
                if r is not None and abs(r["duration"] - expl_dur) < 5:
                    clean_id, clean_title, clean_dur = r["id"], r["title"], r["duration"]
                    break
        save_youtube_info(row.uri, expl_id, clean_id, expl_dur, clean_dur)

0it [00:00, ?it/s]
